# Beer scraper - Zé Delivery

This project scrapes beer prices and other info from the delivery website "Zé Delivery", and returns the cheapest option possible, subject to constraints defined by the user.

# Imports

In [3]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from unidecode import unidecode

import pandas as pd
import numpy as np
import re

# Class creation

In [40]:
def get_url(brand):
    root_url = 'https://www.ze.delivery/produtos/marca/'
    brand = brand.lower()
    brand = unidecode(brand)
    brand = brand.replace("'", "")
    brand = brand.replace(" ", "-")
    return root_url+brand

In [41]:
def handle_price(price):
    price = price.text[3:]
    price = price.replace(',','.')
    return float(price)

In [42]:
def get_mls(product):
    pattern1 = r"(\d+)ml"
    pattern2 = r"(\d*\.*\d+)L"
    try:
        ml = float(re.findall(pattern1, product)[0])
    except:
        try:
            ml = int(float(re.findall(pattern2, product)[0])*1000)
        except:
            ml = np.nan
    return ml

In [84]:
def is_returnable(product):
    if product.find('Apenas o ') == -1:
        return 'No'
    return 'Yes'

In [221]:
class BeerScraper:
    def __init__(self):
        self.driver = None
        self.email = None
        self.password = None
        self.available_brands = []
        self.prices = []
        self.products = []
        self.brands = []
        self.df = None
        # Filters
        self.wanted_brands = []
        self.unwanted_brands = []
        self.returnable = ['Yes','No']
        self.max_mls = 99999
        self.filtered_df = None
        
    def build_driver(self):
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        self.driver = webdriver.Chrome(options=options)
    
    def login(self):
        # Login details
        login_url = 'https://www.ze.delivery/conta/entrar'
        self.email = "brunoprates@poli.ufrj.br" ############### Mudar depois
        self.password = "ze123456"
        
        # Enter login details in form
        self.driver.get(login_url)
        self.driver.implicitly_wait(6)
        self.driver.find_element_by_xpath("""//*[@id="login-mail-input-email"]""").send_keys(self.email)
        self.driver.find_element_by_xpath("""//*[@id="login-mail-input-password"]""").send_keys(self.password)

        # Press sign in button
        button = self.driver.find_element_by_xpath("""//*[@id="login-mail-button-sign-in"]""")
        self.driver.execute_script("arguments[0].click();", button)
        time.sleep(3) # Wait a couple seconds to complete the sign in
        
    def get_available_brands(self):
        url_brands = 'https://www.ze.delivery/produtos/categoria/cervejas'
        self.driver.get(url_brands)
        soup = BeautifulSoup(self.driver.page_source, "html.parser")
        available_brands_html = soup.find_all("h2", class_="css-l9heuk-shelfTitle")
        self.available_brands = [brand_html.text for brand_html in available_brands_html]
        
    def scrape_data(self):
        for brand in self.available_brands:
            #Get page HTML
            url = get_url(brand)
            self.driver.get(url)
            soup = BeautifulSoup(self.driver.page_source, "html.parser")
            
            #Find products and add to instance variable
            products_html = soup.find_all("h3", class_="css-krg860-productTitle")
            for product in products_html:
                self.products.append(product.text)
            
            #Find prices and add to instance variable. 
            prices_html = soup.find_all("div", class_="css-t89dhz-priceText")
            for price in prices_html:
                self.prices.append(handle_price(price))
                self.brands.append(brand) # Leverage the for loop to include brand names
    
    def create_df(self):
        self.df = pd.DataFrame(list(zip(self.products,self.prices,self.brands)),columns=['Product','Price','Brand'])
        self.df['Mls'] = self.df['Product'].map(get_mls)
        self.df['Price Per Liter'] = self.df['Price']/self.df['Mls']*1000
        self.df['Returnable'] = self.df['Product'].map(is_returnable)
        # Sort
        self.df = self.df.sort_values('Price Per Liter')
    
    def set_filters(self,wb=[],ub=[],r=['Yes','No'],mm=99999):
        self.wanted_brands = wb
        self.unwanted_brands = ub
        self.returnable = r
        self.max_mls = mm
        
    def apply_filters(self):
        # Conditions
        c0 = self.df['Brand'].isin(self.wanted_brands) if len(self.wanted_brands)>0 else df['Brand']==df['Brand']
        c1 = np.logical_not(self.df['Brand'].isin(self.unwanted_brands))
        c2 = self.df['Returnable'].isin(self.returnable)
        c3 = self.df['Mls']<=self.max_mls
        
        self.filtered_df = self.df[c0&c1&c2&c3]

In [222]:
a = BeerScraper()

In [223]:
a.build_driver()

In [224]:
a.login()

In [225]:
a.get_available_brands()

In [226]:
a.scrape_data()

In [227]:
a.create_df()

In [212]:
a.set_filters(mm=350,r=['Yes','No'],ub=['Skol','Antarctica'])

In [152]:
a.apply_filters()

In [153]:
a.filtered_df

,Product,Price,Brand,Mls,Price Per Liter,Returnable
20,Skol 350ml,2.29,Skol,350.0,6.542857,No
22,Skol Puro Malte 350ml,2.29,Skol,350.0,6.542857,No
32,Antarctica Pilsen 300ml | Apenas o Líquido,1.99,Antarctica,300.0,6.633333,Yes
55,Bohemia 350ml,2.49,Bohemia,350.0,7.114286,No
37,Antarctica Subzero 269ml,1.99,Antarctica,269.0,7.397770,No
33,Antarctica Pilsen 350ml,2.59,Antarctica,350.0,7.400000,No
12,Brahma Chopp 350ml,2.69,Brahma,350.0,7.685714,No
16,Brahma Malzbier 350ml,2.69,Brahma,350.0,7.685714,No
39,Antarctica Pilsen 269ml,2.08,Antarctica,269.0,7.732342,No
17,Skol 269ml,2.09,Skol,269.0,7.769517,No


# Data Collection

### Build Driver

In [2]:
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(options=options)

### Login to website

In [3]:
# Login url
login_url = 'https://www.ze.delivery/conta/entrar'

In [4]:
# Login details
email = "brunoprates@poli.ufrj.br"
password = "ze123456"

In [5]:
# Enter login details in form
driver.get(login_url)
driver.implicitly_wait(6)
driver.find_element_by_xpath("""//*[@id="login-mail-input-email"]""").send_keys(email)
driver.find_element_by_xpath("""//*[@id="login-mail-input-password"]""").send_keys(password)

# Press sign in button
button = driver.find_element_by_xpath("""//*[@id="login-mail-button-sign-in"]""")
driver.execute_script("arguments[0].click();", button)
time.sleep(3) # Wait a couple seconds to complete the sign in

### Get available brands

In [6]:
def get_available_brands():
    url_brands = 'https://www.ze.delivery/produtos/categoria/cervejas'
    driver.get(url_brands)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    available_brands_html = soup.find_all("h2", class_="css-l9heuk-shelfTitle")
    available_brands = [brand_html.text for brand_html in available_brands_html]
    return available_brands

In [7]:
available_brands = get_available_brands()
available_brands

['Brahma',
 'Skol',
 'Budweiser',
 'Antarctica',
 'Original',
 'Stella Artois',
 "Beck's",
 'Corona',
 'Bohemia',
 'Retornáveis',
 'Serramalte',
 'Colorado',
 'Hoegaarden',
 'Wäls',
 'Leffe',
 'Patagonia',
 'Caracu',
 'Goose Island',
 'Franziskaner',
 'Kona',
 'Cervejaria Bohemia',
 'Farra Bier',
 'Flamingo',
 'Hocus Pocus',
 'Motim',
 'Noi',
 'Overhop',
 'Three Monkeys']

### Get urls to be scraped

In [8]:
def get_url(brand):
    root_url = 'https://www.ze.delivery/produtos/marca/'
    brand = brand.lower()
    brand = unidecode(brand)
    brand = brand.replace("'", "")
    brand = brand.replace(" ", "-")
    return root_url+brand

### Get beer details

In [9]:
def get_prices(prices_html):
    prices = []
    for price in prices_html:
        price = price.text[3:]
        price = price.replace(',','.')
        prices.append(float(price))
    return prices

In [10]:
def get_products(products_html):
    return [product.text for product in products_html]

In [11]:
def OLD_get_mls(products):
    mls=[]
    pattern1 = r"(\d+)ml"
    pattern2 = r"(\d+)L"  # Erra quando eh 1.5L
    for product in products:
        try:
            ml = int(re.findall(pattern1, product)[0])
        except:
            try:
                ml = int(re.findall(pattern2, product)[0])*1000
            except:
                ml = 0#np.Nan
        mls.append(ml)
    return mls

In [77]:
def get_mls(product):
    pattern1 = r"(\d+)ml"
    pattern2 = r"(\d*\.*\d+)L"
    try:
        ml = float(re.findall(pattern1, product)[0])
    except:
        try:
            ml = int(float(re.findall(pattern2, product)[0])*1000)
        except:
            ml = np.nan
    return ml

In [13]:
def get_beers_details(brand):
    url = get_url(brand)
    
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    products_html = soup.find_all("h3", class_="css-krg860-productTitle")
    products = get_products(products_html)
    
    prices_html = soup.find_all("div", class_="css-t89dhz-priceText")
    prices = get_prices(prices_html)
    
    brand_list = [brand for i in range(len(products))]

    return products, prices, brand_list

### Create the Dataframe

In [14]:
def is_returnable(product):
    if product.find('Apenas o ') == -1:
        return 'No'
    return 'Yes'

In [15]:
def create_df(available_brands, num_brands=len(available_brands)):
    total_products = []
    total_prices = []
    total_brands = []
    
    for i in range(num_brands):
        products, prices, brand_list = get_beers_details(available_brands[i])
        total_products += products
        total_prices += prices
        total_brands += brand_list
        
    df = pd.DataFrame(list(zip(total_products,total_prices,total_brands)),columns=['Product','Price','Brand'])
    df['Mls'] = df['Product'].map(get_mls)
    df['Price Per Liter'] = df['Price']/df['Mls']*1000
    df = df.sort_values('Price Per Liter')
    df['Returnable'] = df['Product'].map(is_returnable)
    
    return df

In [16]:
df = create_df(available_brands)

In [17]:
df

,Product,Price,Brand,Mls,Price Per Liter,Returnable
40,Antarctica Subzero 473ml,2.69,Antarctica,473,5.687104,No
35,Antarctica Pilsen 1L | Apenas o líquido,6.09,Antarctica,1000,6.090000,Yes
18,Skol 473ml,2.99,Skol,473,6.321353,No
20,Skol 350ml,2.29,Skol,350,6.542857,No
23,Skol Puro Malte 350ml,2.29,Skol,350,6.542857,No
...,...,...,...,...,...,...
77,Wäls Trippel 375ml,19.99,Wäls,375,53.306667,No
110,Hocus Pocus IPA com Abacaxi 500ml,26.90,Hocus Pocus,500,53.800000,No
129,Overhop Imperial IPA 500ml,26.90,Overhop,500,53.800000,No
109,Hocus Pocus IPA 500ml,27.90,Hocus Pocus,500,55.800000,No


### Define Conditions

In [88]:
wanted_brands = []
unwanted_brands = []
returnable = []
max_mls = None

In [120]:
wanted_brands = []
unwanted_brands = ['Brahma','Skol','Original']
returnable = ['No']
max_mls = 270

In [93]:
df = df2

In [95]:
df

,Product,Price,Brand
0,Brahma Chopp 300ml | Apenas o Líquido,2.39,Brahma
1,Brahma Duplo Malte 330ml,3.59,Brahma
2,Brahma Duplo Malte 350ml,3.19,Brahma
3,Brahma Duplo Malte 600ml | Apenas o Líquido,6.89,Brahma
4,Brahma Duplo Malte 600ml | Vasilhame Incluso,7.89,Brahma
...,...,...,...
131,Three Monkeys American IPA 500ml,22.80,Three Monkeys
132,Three Monkeys White IPA 500ml,21.60,Three Monkeys
133,"Three Monkeys Sour Ale de Pitaya, Framboesa e ...",24.00,Three Monkeys
134,Three Monkeys American Sour Ale de Pitaya e Go...,24.00,Three Monkeys


In [94]:
# Conditions
c0 = df['Brand'].isin(wanted_brands) if len(wanted_brands)>0 else True
c1 = np.logical_not(df['Brand'].isin(unwanted_brands))
c2 = df['Returnable'].isin(returnable)
c3 = df['Mls']<=max_mls

KeyError: 'Returnable'

In [123]:
df[c0&c1&c2&c3]

,Product,Price,Brand,Mls,Price Per Liter,Returnable
39,Antarctica Subzero 269ml,1.99,Antarctica,269,7.397770,No
41,Antarctica Pilsen 269ml,2.08,Antarctica,269,7.732342,No
61,Bohemia 269ml,2.11,Bohemia,269,7.843866,No
27,Budweiser 269ml,2.60,Budweiser,269,9.665428,No
46,Stella Artois 269ml,2.80,Stella Artois,269,10.408922,No
56,Coronita 210ml,3.79,Corona,210,18.047619,No


In [24]:
df[df['Returnable']=='Yes']

,Product,Price,Brand,Mls,Price Per Liter,Returnable
35,Antarctica Pilsen 1L | Apenas o líquido,6.09,Antarctica,1000,6.090000,Yes
33,Antarctica Pilsen 300ml | Apenas o Líquido,1.99,Antarctica,300,6.633333,Yes
14,Brahma Chopp 1L | Apenas o Líquido - Unidade,7.09,Brahma,1000,7.090000,Yes
0,Brahma Chopp 300ml | Apenas o Líquido,2.39,Brahma,300,7.966667,Yes
60,Bohemia 300ml | Apenas o Líquido,2.39,Bohemia,300,7.966667,Yes
36,Antarctica Pilsen 600ml | Apenas o Líquido,5.29,Antarctica,600,8.816667,Yes
22,Skol 600ml | Apenas o Líquido,5.69,Skol,600,9.483333,Yes
7,Brahma Chopp 600ml | Apenas o Líquido - Unidade,6.29,Brahma,600,10.483333,Yes
24,Skol Puro Malte 600ml | Apenas o Líquido,6.29,Skol,600,10.483333,Yes
58,Bohemia 600ml | Apenas o Líquido,6.59,Bohemia,600,10.983333,Yes


In [25]:
df[df['Brand']=='Brahma']

,Product,Price,Brand,Mls,Price Per Liter,Returnable
10,Brahma Chopp 269ml,1.87,Brahma,269,6.951673,No
14,Brahma Chopp 1L | Apenas o Líquido - Unidade,7.09,Brahma,1000,7.090000,Yes
9,Brahma Chopp 350ml,2.69,Brahma,350,7.685714,No
17,Brahma Malzbier 350ml,2.69,Brahma,350,7.685714,No
5,Brahma Chopp 473ml,3.69,Brahma,473,7.801268,No
0,Brahma Chopp 300ml | Apenas o Líquido,2.39,Brahma,300,7.966667,Yes
2,Brahma Duplo Malte 350ml,3.19,Brahma,350,9.114286,No
12,Brahma Chopp 355ml - Unidade,3.29,Brahma,355,9.267606,No
6,Brahma Malzbier 355ml,3.29,Brahma,355,9.267606,No
8,Brahma Zero 350ml - Unidade,3.39,Brahma,350,9.685714,No
